In [10]:
import requests
import subprocess
import sys
import os
import tarfile
import time

In [11]:
def get_top_pypi_packages(url='https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json'):
    response = requests.get(url)
    response.raise_for_status() 
    
    data = response.json()
    packages = [package['project'] for package in data['rows'][:1500]]

    return packages

top_packages = get_top_pypi_packages()
print(top_packages)


['boto3', 'botocore', 'urllib3', 'requests', 'wheel', 'certifi', 'idna', 'typing-extensions', 'charset-normalizer', 'pip', 'setuptools', 'python-dateutil', 's3transfer', 'packaging', 'aiobotocore', 'pyyaml', 'six', 's3fs', 'fsspec', 'numpy', 'cryptography', 'grpcio-status', 'google-api-core', 'cffi', 'pycparser', 'pypular', 'pandas', 'importlib-metadata', 'pyasn1', 'rsa', 'zipp', 'attrs', 'click', 'pydantic', 'protobuf', 'jmespath', 'platformdirs', 'pytz', 'jinja2', 'colorama', 'markupsafe', 'pyjwt', 'awscli', 'tomli', 'wrapt', 'google-auth', 'googleapis-common-protos', 'filelock', 'cachetools', 'requests-oauthlib', 'oauthlib', 'pluggy', 'virtualenv', 'pyarrow', 'docutils', 'jsonschema', 'pyasn1-modules', 'pytest', 'exceptiongroup', 'aiohttp', 'pyparsing', 'sqlalchemy', 'scipy', 'isodate', 'multidict', 'psutil', 'pyopenssl', 'yarl', 'iniconfig', 'decorator', 'soupsieve', 'pygments', 'tzdata', 'async-timeout', 'beautifulsoup4', 'frozenlist', 'aiosignal', 'tqdm', 'grpcio', 'pillow', 'req

In [15]:
def dl_packages(packages, target_directory):
    count = 0
    for package in packages:
        package_dir = os.path.join(target_directory, package)
        os.makedirs(package_dir, exist_ok=True)
        retries = 3
        delay = 1
        while retries > 0:
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "download", package, "--no-binary=:all:", "--no-deps", "--dest", package_dir])
                print(f"Downloaded {package} into {package_dir}")
                count += 1
                print(f"Progress: {count}/{len(packages)}")
                break  # Break out of the retry loop if download is successful
            except subprocess.CalledProcessError as e:
                print(f"Failed to download {package}: {e}")
                retries -= 1
                if retries == 0:
                    print(f"Max retries reached for {package}. Skipping.")
                else:
                    print(f"Retrying in {delay} seconds...")
                    time.sleep(delay)
                    delay *= 2  # Exponential backoff
                    continue

top_packages = get_top_pypi_packages()

download_directory = '/mnt/volume_nyc1_01/benignPyPI/'

dl_packages(top_packages, download_directory)

  Using cached boto3-1.34.136.tar.gz (108 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Saved /mnt/volume_nyc1_01/benignPyPI/boto3/boto3-1.34.136.tar.gz
Successfully downloaded boto3
Downloaded boto3 into /mnt/volume_nyc1_01/benignPyPI/boto3
Progress: 1/1500
  Using cached botocore-1.34.136.tar.gz (12.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Saved /mnt/volume_nyc1_01/benignPyPI/botocore/botocore-1.34.136.tar.gz
Su

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [91 lines of output]
        Using cached meson_python-0.16.0.tar.gz (82 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Installing backend dependencies: started
        Installing backend dependencies: finished with status 'error'
        error: subprocess-exited-with-error
      
        × pip subprocess to install backend dependencies did not run successfully.
        │ exit code: 1
        ╰─> [68 lines of output]
              Using cached ninja-1.11.1.1.tar.gz (132 kB)
              Installing build dependencies: started
              Installing build dependencies: finished with status 'done'
              Getting requirements to build wheel: 

Failed to download numpy: Command '['/bin/python3', '-m', 'pip', 'download', 'numpy', '--no-binary=:all:', '--no-deps', '--dest', '/mnt/volume_nyc1_01/benignPyPI/numpy']' returned non-zero exit status 1.
Retrying in 1 seconds...
  Using cached numpy-2.0.0.tar.gz (18.3 MB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/req_command.py", line 245, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_internal/commands/download.py", line 132, in run
    requirement_set = resolver.resolve(reqs, check_supported_wheels=True)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
    

Failed to download numpy: Command '['/bin/python3', '-m', 'pip', 'download', 'numpy', '--no-binary=:all:', '--no-deps', '--no-build-isolation', '--dest', '/mnt/volume_nyc1_01/benignPyPI/numpy']' returned non-zero exit status 2.
Retrying in 2 seconds...
  Using cached numpy-2.0.0.tar.gz (18.3 MB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


...

ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_internal/cli/req_command.py", line 245, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_internal/commands/download.py", line 132, in run
    requirement_set = resolver.resolve(reqs, check_supported_wheels=True)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
    

Failed to download numpy: Command '['/bin/python3', '-m', 'pip', 'download', 'numpy', '--no-binary=:all:', '--no-deps', '--no-build-isolation', '--dest', '/mnt/volume_nyc1_01/benignPyPI/numpy']' returned non-zero exit status 2.
Max retry attempts reached for numpy. Skipping...
  Using cached cryptography-42.0.8.tar.gz (671 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [72 lines of output]
        Using cached setuptools-70.1.1-py3-none-any.whl
        Using cached wheel-0.43.0-py3-none-any.whl
        Using cached cffi-1.16.0.tar.gz (512 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached setuptools_rust-1.9.0-py3-none-any.whl
        Using cached pycparser-2.22-py3-none-any.whl
        Using cached semantic_version-2.10.0-py2.py3-none-any.whl
        error: subprocess-exited-with-error
      
        × Building wheel for cffi (pyproject.toml) did not run successfully.
        │ e

Failed to download cryptography: Command '['/bin/python3', '-m', 'pip', 'download', 'cryptography', '--no-binary=:all:', '--no-deps', '--dest', '/mnt/volume_nyc1_01/benignPyPI/cryptography']' returned non-zero exit status 1.
Retrying in 1 seconds...
  Using cached cryptography-42.0.8.tar.gz (671 kB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Saved /mnt/volume_nyc1_01/benignPyPI/cryptography/cryptography-42.0.8.tar.gz
Successfully downloaded cryptography


KeyboardInterrupt: 

In [ ]:
repo_dir = "/mnt/volume_nyc1_01/benignPyPI/"
#Find all tar.gz files in the repository
tar_files = []
for root, dirs, files in os.walk(repo_dir):
    for file in files:
        if file.endswith(".tar.gz") or file.endswith(".tar.bz2") or file.endswith(".tar.xz"):
            tar_files.append(os.path.join(root, file))

#Unzip the files with different compression methods
for tar_file in tar_files:
    if tar_file.endswith(".tar.gz"):
        mode = 'r:gz'
    elif tar_file.endswith(".tar.bz2"):
        mode = 'r:bz2'
    elif tar_file.endswith(".tar.xz"):
        mode = 'r:xz'
    else:
        mode = 'r'
    
    try:
        with tarfile.open(tar_file, mode) as tar_ref:
            extract_dir = os.path.splitext(os.path.splitext(tar_file)[0])[0]
            tar_ref.extractall(extract_dir)
            print(f"Extracted {tar_file} to {extract_dir}")
    except (tarfile.ReadError, tarfile.CompressionError) as e:
        print(f"Failed to extract {tar_file}: {e}")

print("Extraction process completed.")